In [ ]:
import numpy as np
import pandas as pd

import statistics
import random

from statsmodels.stats.outliers_influence import variance_inflation_factor
from imblearn.over_sampling import SMOTE

from sklearn.metrics import fbeta_score
from sklearn import metrics

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
q1features_trainvalid = pd.read_pickle("q1features_trainvalid.pickle")
q1target_trainvalid = pd.read_pickle("q1target_trainvalid.pickle")
q1features_test = pd.read_pickle("q1features_test.pickle")
q1target_test = pd.read_pickle("q1target_test.pickle")

Attributes with zero variance were removed from the model; these attributes did not contribute any information to the model.

In [ ]:
#Removing zero variance attributes (training set)
q1features_trainvalid = q1features_trainvalid.loc[:, q1features_trainvalid.nunique(axis=0) != 1]

In [ ]:
#Removing zero variance attributes (test set)
q1features_test = q1features_test.loc[:, q1features_test.nunique(axis=0) != 1]

Attributes with more than 10% missing data were dropped from the dataset. Dr. Iris Eekhout and colleagues in their 2013 paper titled "Missing data in a multi-item instrument were best handled by multiple imputation at the item score level" (https://doi.org/10.1016/j.jclinepi.2013.09.009) found that single-imputation methods can result in biased estimates when the dataset has a higher proportion of missing data. Single-imputation methods will be applied later in the analysis, hence the removal of these attributes with a high proportion of missing data at this step.

In [ ]:
#Removing attributes with more than 10% missing data (training set)
for col in q1features_trainvalid.columns.tolist():
  if q1features_trainvalid[col].isna().sum()/len(q1features_trainvalid[col]) > 0.10:
    q1features_trainvalid = q1features_trainvalid.drop([col], axis=1)

In [ ]:
#Removing attributes with more than 10% missing data (test set)
for col in q1features_test.columns.tolist():
  if q1features_test[col].isna().sum()/len(q1features_test[col]) > 0.10:
    q1features_test = q1features_test.drop([col], axis=1)

Imputation of missing data was performed. The alternative was to use complete case analysis, which involves removing from the dataset any rows that were missing any attributes. Complete case analysis leads to a loss of information and can lead to bias if the missing data is not missing completely at random.

Single-imputation methods were used; missing categorical values were imputed using the mode of that attribute and missing numeric values were imputed using the median. The test data set was imputed using the mode and median values of the training data set. To avoid data leakage, the training data set was not imputed using test set data.


In [ ]:
#Imputing test set
for col in q1features_test.columns.tolist():
  q1features_test[col] = q1features_test[col].replace([None], np.nan)
  if (q1features_test[col].dtype == 'category' or q1features_trainvalid[col].dtype =='datetime64[ns]'):
    q1features_test[col]= q1features_test[col].fillna(random.choice(statistics.multimode(q1features_trainvalid[col])))
  if (q1features_test[col].dtype == 'Int64' or q1features_test[col].dtype == 'int64'):
    q1features_test[col] = q1features_test[col].astype('float64')
  if (q1features_test[col].dtype == 'float64'):
    q1features_test[col] = q1features_test[col].fillna(q1features_trainvalid[col].median())

In [ ]:
#Imputing training set
for col in q1features_trainvalid.columns.tolist():
  q1features_trainvalid[col] = q1features_trainvalid[col].replace([None], np.nan)
  if (q1features_trainvalid[col].dtype == 'category' or q1features_trainvalid[col].dtype =='datetime64[ns]'):
    q1features_trainvalid[col]= q1features_trainvalid[col].fillna(random.choice(statistics.multimode(q1features_trainvalid[col])))
  if (q1features_trainvalid[col].dtype == 'Int64' or q1features_trainvalid[col].dtype == 'int64'):
    q1features_trainvalid[col] = q1features_trainvalid[col].astype('float64')
  if (q1features_trainvalid[col].dtype == 'float64'):
    q1features_trainvalid[col] = q1features_trainvalid[col].fillna(q1features_trainvalid[col].median())

Numeric attributes were standardized; variable importance will be assessed using regression models, and standardization allows for easier interpretation of the model coefficients. Standardization was performed instead of normalization, as standardization is more resistant to outliers.

In [ ]:
#Standardization  of test set
for col in q1features_test.columns.tolist():
  if (q1features_test[col].dtype == 'Int64' or q1features_test[col].dtype == 'int64' or q1features_test[col].dtype == 'float64'):
    for i in range(len(q1features_test[col])):
      q1features_test.loc[q1features_test.index[i], col] = (q1features_test.loc[q1features_test.index[i], col] - q1features_trainvalid[col].mean())/np.std(q1features_trainvalid[col])

In [ ]:
#Standardization of training set
for col in q1features_trainvalid.columns.tolist():
  if (q1features_trainvalid[col].dtype == 'Int64' or q1features_trainvalid[col].dtype == 'int64' or q1features_trainvalid[col].dtype == 'float64'):
    for i in range(len(q1features_trainvalid[col])):
      q1features_trainvalid.loc[q1features_trainvalid.index[i], col] = (q1features_trainvalid.loc[q1features_trainvalid.index[i], col] - q1features_trainvalid[col].mean())/np.std(q1features_trainvalid[col])

In [ ]:
#The SMOTE algorithm cannot handle datetime, so converting to number of
#days since January 1st, year one
for i in range(len(q1features_trainvalid['INTERVIEWDATE'])):
  q1features_trainvalid.loc[q1features_trainvalid.index[i], 'INTERVIEWDATE'] =  q1features_trainvalid.loc[q1features_trainvalid.index[i], 'INTERVIEWDATE'].toordinal()

for i in range(len(q1features_test['INTERVIEWDATE'])):
  q1features_test.loc[q1features_test.index[i], 'INTERVIEWDATE'] =  q1features_test.loc[q1features_test.index[i], 'INTERVIEWDATE'].toordinal()

q1features_trainvalid['INTERVIEWDATE'] = q1features_trainvalid['INTERVIEWDATE'].astype('float64')
q1features_test['INTERVIEWDATE'] = q1features_test['INTERVIEWDATE'].astype('float64')

In [ ]:
#Converting categorical variables to dummy variables
for col in q1features_trainvalid.columns.tolist():
  if (q1features_trainvalid[col].dtype == 'category'):
    q1features_trainvalid = pd.concat([q1features_trainvalid, pd.get_dummies(q1features_trainvalid[col], prefix=col, drop_first=True, dtype='float')], axis=1)
    q1features_trainvalid = q1features_trainvalid.drop([col], axis=1)

In [ ]:
for col in q1features_test.columns.tolist():
  if (q1features_test[col].dtype == 'category'):
    q1features_test = pd.concat([q1features_test, pd.get_dummies(q1features_test[col], prefix=col, drop_first=True, dtype='float')], axis=1)
    q1features_test = q1features_test.drop([col], axis=1)

In [ ]:
q1target_trainvalid = q1target_trainvalid.astype('bool')
q1target_trainvalid = q1target_trainvalid.astype('float')

In [ ]:
q1target_test = q1target_test.astype('bool')
q1target_test = q1target_test.astype('float')

Multicollinearity can lead to model overfitting and can artificially hide the importance of explanatory variables. Multicollinearity (and collinearity) can be assessed using variance inflation factor values. A VIF value of over 10 indicates serious multicollinearity (Vittinghoff, E., Shiboski, S., Glidden, D., & McCulloch, C. (2004). Regression Methods in Biostatistics: Linear, Logistic, Survival and Repeated Measures Models. New York:Springer. https://doi.org/10.1007/b138825). Independent variables were iteratively removed from the model starting with the variable with the highest VIF value until all VIF values were below an accceptable threshold.

In [ ]:
#Removing highly correlated variables
unwanted = q1features_trainvalid.columns[q1features_trainvalid.columns.str.startswith('IDAY')]
q1features_trainvalid.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_trainvalid.columns[q1features_trainvalid.columns.str.startswith('QSTVER')]
q1features_trainvalid.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_trainvalid.columns[q1features_trainvalid.columns.str.startswith('STATE')]
q1features_trainvalid.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_trainvalid.columns[q1features_trainvalid.columns.str.startswith('_AGE65YR')]
q1features_trainvalid.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_trainvalid.columns[q1features_trainvalid.columns.str.startswith('IMONTH')]
q1features_trainvalid.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_trainvalid.columns[q1features_trainvalid.columns.str.startswith('INTERVIEWDATE')]
q1features_trainvalid.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_trainvalid.columns[q1features_trainvalid.columns.str.startswith('_AGE_G')]
q1features_trainvalid.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_trainvalid.columns[q1features_trainvalid.columns.str.startswith('_AGE80')]
q1features_trainvalid.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_trainvalid.columns[q1features_trainvalid.columns.str.startswith('HTM4')]
q1features_trainvalid.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_trainvalid.columns[q1features_trainvalid.columns.str.startswith('WTKG3')]
q1features_trainvalid.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_trainvalid.columns[q1features_trainvalid.columns.str.startswith('USENOW3')]
q1features_trainvalid.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_trainvalid.columns[q1features_trainvalid.columns.str.startswith('EDUCA')]
q1features_trainvalid.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_trainvalid.columns[q1features_trainvalid.columns.str.startswith('SLEPTIM1')]
q1features_trainvalid.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_trainvalid.columns[q1features_trainvalid.columns.str.startswith('PRIMINSR')]
q1features_trainvalid.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_trainvalid.columns[q1features_trainvalid.columns.str.startswith('FMONTH')]
q1features_trainvalid.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_trainvalid.columns[q1features_trainvalid.columns.str.startswith('SMOKE100')]
q1features_trainvalid.drop(unwanted, axis=1, inplace=True)

In [ ]:
unwanted = q1features_test.columns[q1features_test.columns.str.startswith('IDAY')]
q1features_test.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_test.columns[q1features_test.columns.str.startswith('QSTVER')]
q1features_test.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_test.columns[q1features_test.columns.str.startswith('STATE')]
q1features_test.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_test.columns[q1features_test.columns.str.startswith('_AGE65YR')]
q1features_test.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_test.columns[q1features_test.columns.str.startswith('IMONTH')]
q1features_test.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_test.columns[q1features_test.columns.str.startswith('INTERVIEWDATE')]
q1features_test.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_test.columns[q1features_test.columns.str.startswith('_AGE_G')]
q1features_test.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_test.columns[q1features_test.columns.str.startswith('_AGE80')]
q1features_test.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_test.columns[q1features_test.columns.str.startswith('HTM4')]
q1features_test.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_test.columns[q1features_test.columns.str.startswith('WTKG3')]
q1features_test.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_test.columns[q1features_test.columns.str.startswith('USENOW3')]
q1features_test.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_test.columns[q1features_test.columns.str.startswith('EDUCA')]
q1features_test.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_test.columns[q1features_test.columns.str.startswith('SLEPTIM1')]
q1features_test.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_test.columns[q1features_test.columns.str.startswith('PRIMINSR')]
q1features_test.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_test.columns[q1features_test.columns.str.startswith('FMONTH')]
q1features_test.drop(unwanted, axis=1, inplace=True)

unwanted = q1features_test.columns[q1features_test.columns.str.startswith('SMOKE100')]
q1features_test.drop(unwanted, axis=1, inplace=True)

The dataset is imbalanced; a majority of respondents reported that they did not have COVID-19 symptoms that lasted longer than 3 months. Imbalanced data adversely affects model performance; if the model encounters few instances of the minority class then it will be unable to effectively learn from this class.

The SMOTE algorithm was used to oversample the minority class; this address the issue of imbalanced data.

In [ ]:
#SMOTE Algorithm
from imblearn.over_sampling import SMOTE
smo = SMOTE(random_state = 2, k_neighbors=10)
q1features_trainvalid_SM, q1target_trainvalid_SM = smo.fit_resample(q1features_trainvalid, q1target_trainvalid.ravel())

In [ ]:
f2_list = []

Logistic Regression

In [ ]:
logr = LogisticRegression(random_state=1, solver="liblinear" )
logReg =logr.fit(q1features_trainvalid_SM, q1target_trainvalid_SM)
logRegPrediction = logReg.predict(q1features_test)
logRegPrediction = np.where(logRegPrediction > 0.5, 1, 0)
f2_score = fbeta_score(q1target_test, logRegPrediction, beta=2)
f2_score

In [ ]:
logRegMatrix = metrics.confusion_matrix(q1target_test, logRegPrediction)
logRegMatrix

In [ ]:
#The LogisticRegression function of the sklearn library does not
#support retrieving the p-values of each independent variable directly.
#Using the statsmodels library to retrieve this information
import statsmodels.api as sm
log_reg = sm.Logit(q1target_trainvalid_SM, q1features_trainvalid_SM).fit()
print(log_reg.summary())

Naive Bayes

In [ ]:
# Build a Gaussian Classifier
model = GaussianNB()

# Model training
nbModel = model.fit(q1features_trainvalid_SM, q1target_trainvalid_SM)

#Making predictions on the validation set
nbPrediction = nbModel.predict(q1features_test)

f2_score = fbeta_score(q1target_test, nbPrediction, beta=2)

f2_score

In [ ]:
nbMatrix = metrics.confusion_matrix(q1target_test, nbPrediction)
nbMatrix

Random Forest

In [ ]:
rf = RandomForestClassifier(n_estimators=100, max_depth=1, criterion="log_loss", random_state=42)
rf.fit(q1features_trainvalid_SM, q1target_trainvalid_SM)
rfPrediction = rf.predict(q1features_test)
f2_score = fbeta_score(q1target_test, rfPrediction, beta=2)
f2_score

In [ ]:
rfMatrix = metrics.confusion_matrix(q1target_test, rfPrediction)
rfMatrix

Gradient-Boosted Trees

In [ ]:
bst = GradientBoostingClassifier(n_estimators=25, max_depth=1, subsample=0.2, random_state=42).fit(q1features_trainvalid_SM, q1target_trainvalid_SM)
bstPrediction = bst.predict(q1features_test)
f2_score = fbeta_score(q1target_test, bstPrediction, beta=2)
f2_score

In [ ]:
bstMatrix = metrics.confusion_matrix(q1target_test, bstPrediction)
bstMatrix

From this initial code, the logistic regression model seems to have the best performance. However, the F-2 score is quite low (apprx. 0.49). Among the next steps of this data analysis project will be to further optimize model performance.